In [ ]:
'''loadData.ipynb : Make Pandas table of data to be called from h_DsRnn_Prep_sba.ipynb

PREC: h_DsRnn_ExTb.ipynb
AUTH: Hendrik.Mandelkow@gmail.com
''';

# Imports

In [ ]:
import os
import pandas as pd
pd.set_option("display.max_colwidth",16)

# Requirements - for testing

In [ ]:
try: ( mkRNN )
except:
    # %run ./h_DsRnn_def.ipynb
    %run ./modelDef.ipynb

try: ( ProcTb, Par )
except:
    # %run ./h_DsRnn_ProcTb.ipynb
    %run ./trainPars.ipynb
ProcTb.shape


# mkExTb: 1st find all experiments in DataDir
 1. Start with a set of metapars (created in h_DsRnn_Prep)
 2. Find all the Par.NiiFile in Par.DataDir and create a pd.DataFrame column - one row per experiment (dataset).
 3. Populate other columns with derived metadata:
    - NiiSz
    - ProcDir
    - ExId
    - Su
    - Se
    - Ex
    - ExRank ???
    - ValRank ???
    - ValTag : (str) 'ValEx'
    - Sc : section ???
    - HypnoFile : Sleep score file
    - VmrkFile : EEG marker file
    - ...
 4. Load and preproc data to populate columns:
    - Data
    - ...


In [ ]:
Par = ddict(Par) # +++ Par was created in h_DsRnn_Prep
os.chdir(Par.DataDir)
print(os.getcwd())
#< Fname = hsys('find Su%s -iname "%s"'%( Par.SuId, Par.NiiFile) ) # MoCorr + MoReg
Fname = hsys('find sub-%s -iname "%s"'%( Par.SuId, Par.NiiFile) ) # MoCorr + MoReg
# print(Fname[:10],sep='\n')
print('\n'.join(Fname[:10]))
print('...%u'%len(Fname))

In [ ]:
ExTb = pd.DataFrame(Fname,columns=['NiiFile'])
ExTb['NiiSz'] = ExTb.NiiFile.apply(os.path.getsize) # +++ size in bytes
ExTb['NiiSz'] = ExTb['NiiFile'].apply( lambda x: niba.load(x).shape[-1] ) # +++ size in volumens, much slower!
ExTb = ExTb.sort_values('NiiSz').reset_index(drop=True)
# ExTb

In [ ]:
### mk ProcDir + NiiFile
#< ExTb['ProcDir'] = Tb.NiiFile.apply(hFpath)
# HOWTO extract (multiple) regex groups and store them in new columns:
ExTb[['ProcDir','NiiFile']] = ExTb.NiiFile.str.extract(r'(.*/)([^/]+)') # 2 captures for 2 columns


```python
# HOWTO use Pandas rename which requires a mapper function or a dict-like:
tmp = ExTb.NiiFile.str.extract(r'(.*/)([^/]+)').rename(columns={0:'ProcDir',1:'NiiFile'})
tmp = ExTb.NiiFile.str.extract(r'(.*/)([^/]+)').rename(columns=dict(zip(range(2),['ProcDir','NiiFile'])))
tmp = ExTb.NiiFile.str.extract(r'(.*/)([^/]+)').rename(columns=lambda n:('ProcDir','NiiFile')[n])
tmp = ExTb.NiiFile.str.extract(r'(?P<ProcDir>.*/)(?P<NiiFile>[^/]+)')
tmp
ExTb[['ProcDir','NiiFile']] = tmp
# pd.concat([tmp,ExTb],1)
```

In [ ]:
### mk Su,Se,Ex,ExId (='20180706_0423' eg)
# HOWTO extract (multiple) regex groups and store them in new columns:
# tmp = ExTb.ProcDir.str.extract(r'^Su(\d+)/Se(\d+)/Ex(\d+)/([\d_]+).results')
# tmp = ExTb.ProcDir.str.extract(r'^Su(\d+)/Se(\d+)/Ex(\d+)/').astype(int).rename(columns={0:'Su',1:'Se',2:'Ex'})
# tmp = ExTb.ProcDir.str.extract(r'^Su(\d+)/Se(\d+)/Ex(\d+)/').astype(int); tmp.columns = ['Su','Se','Ex']
# tmp = ExTb.ProcDir.str.extract(r'^Su(?P<Su>\d+)/Se(?P<Se>\d+)/Ex(?P<Ex>\d+)/').astype(int)
tmp = ExTb.ProcDir.str.extract(r'^sub-(?P<Su>\d+)/ses-(?P<Se>\d+)/run-(?P<Ex>[\d_]+)/').astype(int)
tmp['ExId'] = ExTb.ProcDir.str.extract(r'/([\d_]+).results')
ExTb = pd.concat([tmp, ExTb],1)
## This makes it harder to prepend columns:
# ExTb[['Su','Se','Ex']] = ExTb.ProcDir.str.extract(r'^Su(\d+)/Se(\d+)/Ex(\d+)/').astype(int);
# ExTb['ExId'] = ExTb.ProcDir.str.extract(r'/([\d_]+).results')
# ExTb = pd.concat([ExTb.iloc[-4:],ExTb.iloc[0:-4]]) # sth like this should work

ExTb

In [ ]:
# HOWTO: Move an item in a list:
hmoveitem = lambda L,A,B,n=0: L.insert( L.index(B)+n, L.pop( L.index(A)))
# HOWTO: Move a column in a Pandas table:
hmovecol = lambda df,A,B,n=0: df.insert( list(df).index(B)+n, A, df.pop(A))
hmovecol = lambda df,A,B,n=0: df.insert( df.columns.get_loc(B)+n, A, df.pop(A))

ExTb.sort_values( ['Su','NiiSz'], inplace=True) # sort by Su and size!
ExTb['ExRank'] = ExTb.groupby('Su')['NiiSz'].rank(method='first').astype(int)
hmovecol(ExTb,'ExRank','Ex',1)


## Add Su1Vx1MedianMetric.csv

#### NOTES

* pd.concat is the main function
* pd.append is shorthand for concat rows
* pd.merge is similar (equvalent?) to concat, but mimics SQL
* pd.join is shorthand for merge with certain options.


**************************************
Obsolete given above.

---------------------------------------------

## mk ExTb.ValTag
mk ExTb.ValTag = 'ValSu' if ExTb.Su in Par.ValSu or 'ValEx' if shortest scan for each subj


In [ ]:
#%% 1st, mk new ValTag column:
def ExTb_ValTag( ExTb, Par):
    # HOWTO delete a Pandas column in place without errors:
    ExTb.drop( columns='ValTag', errors='ignore', inplace=True) # remove old column
    ExTb.insert( 3, 'ValTag', False) # +++

    if Par.ValRank:
        ExTb['ExRank'] = ExTb.ValRank # +++
        print('+ Using ExRank = ValRank')
    if Par.ValEx1:
        try: tmp = list(Par.ValEx1)
        except: tmp = [ Par.ValEx1 ]
        assert not any( np.array(tmp) > ExTb['ExRank'].max() ), 'Oops! Par.ValEx1 out of bounds.' 
        tmp = ExTb.ExRank.isin( tmp ) #.to_numpy()?
        assert tmp.any(), 'Oops! ValEx1 out of bounds. Stop here.'
        ExTb.loc[ tmp, 'ValTag'] = 'ValEx'

    if Par.ValSu:
        # ExTb.ValTag[ ExTb.Su.apply(lambda x: x in Par.ValSu) ] = 'ValSu'
        ExTb.ValTag[ ExTb.Su.isin( Par.ValSu) ] = 'ValSu' # .eq() .isin() .notnull() ~ &

In [ ]:
#%% 1st, mk new ValTag column:
# HOWTO delete a Pandas column in place without errors:
ExTb.drop( columns='ValTag', errors='ignore', inplace=True) # remove old column
ExTb.insert( 3, 'ValTag', False) # +++

In [ ]:
if Par.ValRank:
    ExTb['ExRank'] = ExTb.ValRank # +++
    print('+ Using ExRank = ValRank')
if Par.ValEx1:
    try: tmp = list(Par.ValEx1)
    except: tmp = [ Par.ValEx1 ]
    assert not any( np.array(tmp) > ExTb['ExRank'].max() ), 'Oops! Par.ValEx1 out of bounds.' 
    tmp = ExTb.ExRank.isin( tmp ) #.to_numpy()?
    assert tmp.any(), 'Oops! ValEx1 out of bounds. Stop here.'
    ExTb.loc[ tmp, 'ValTag'] = 'ValEx'

In [ ]:
if Par.ValSu:
    # ExTb.ValTag[ ExTb.Su.apply(lambda x: x in Par.ValSu) ] = 'ValSu'
    ExTb.ValTag[ ExTb.Su.isin( Par.ValSu) ] = 'ValSu' # .eq() .isin() .notnull() ~ &

In [ ]:
ExTb.head()

In [ ]:
if False: # TODO: Test for SB_TASK
    warn('+++ TEST avoid out of memory!')
    # ExTb = ExTb[0:ExTb.shape[0]//2]
    ExTb = ExTb[0:20]


## load all Gsig files into dict (Pars) and append to ExTb


-------------------------------------------------------------
The above strategy using pd.merge is complicated and unreliable.

Better to convert to/from list of dicts (records).

In [ ]:
Pars = ExTb.to_dict('records')

# Add vars from Gsig.mat, see h_DsRnn_Gsig_sba
# ['DataDir', 'Fs', 'Gsig', 'GsigLegendre', 'MatFile', 'McPar', 'McParFile', 'McRef', 'NiiDims', 'NiiFile', 'NiiMaskFile', 'Phy', 'PhyFile', 'ProcDir', 'SbJob', 'SbTask', 'SuId', 'Svd']
for par in Pars: # Pars = list of experiments (scans) [ExTb,Exs]
    # print('+ Load: '+par['ProcDir']+Par.GsigFile)
    tmp = par.copy()
    par.update( hdf5.loadmat(par['ProcDir']+Par.GsigFile) ) # +++
    # par.update( ExId = re.sub( '/run-([\d_]+)/', '\1', par.ProcDir))
    # par.update( Hypno= np.loadtxt(par['ProcDir']+'../Hypno.txt'), HypnoFile='../Hypno.txt')
    par.update(tmp)
    print('.',end='')
print(len(Pars))


In [ ]:
ExTb = pd.DataFrame(Pars)

In [ ]:
ExTb['HypnoFile'] = '../Hypno.txt'
ExTb['Hypno'] = ExTb.apply(lambda row: np.loadtxt(row.ProcDir+row.HypnoFile), 'columns')
# TODO: use max_rows?

In [ ]:
print( ExTb.columns )
ExTb.head()

--------------------------------------------

## Preprocess data in ExTb
 * merge channels and ...
 * divide into equal length sections
 
#### SEE: [h_DsRnn_v6b1_sba.ipynb](./h_DsRnn_v6b1_sba.ipynb#mkPars)

In [ ]:
#< Par.setdefault('Fs',20.0)
#< Par.setdefault('TR',3.0)
#< Par.setdefault('TRFS',int(Par['Fs']*Par['TR']))
def mkData(row):
    McPar = row.McPar # *** motion parameters
    McPar = np.r_[ 0*McPar[:1,:], np.diff( McPar, axis=0 )] # *** Diff
    McPar = np.c_[ McPar, np.linalg.norm(McPar,2,-1,True)] # *** add McNorm
    
    # For convenience, let's just append other suff to McPar:
    # print( ( McPar.shape, row.Hypno.shape))
    if McPar.shape[0] != row.Hypno.size:
        warn('Size mismatch. Check this.')
        # row.Hypno.resize( McPar.shape[0], refcheck=False)
        # row.Hypno = np.append(row.Hypno,0*McPar[row.Hypno.size:])[:McPar.shape[0]]
    # elseif True:
    # McPar = np.c_[ McPar, row.Hypno ] # *** add sleep scores
    McPar = np.c_[ McPar, np.append(row.Hypno,0*McPar[row.Hypno.size:])[:McPar.shape[0]] ] # *** add sleep scores

    McPar = np.c_[ McPar, 1+0*McPar[:,:1] ] # *** add Mask

    if Par['Xnul'] is not None:
        row.Phy[:,Par['Xnul']] = 0.0 # +++ null chans for dropout purposes

    Data = hXtv2Data( row.Phy, np.c_[ McPar, row.Gsig], Par.TRFS, MaskVal= Par.MaskVal) # ++++
    
    xi = row.Phy.shape[-1]
    xi = slice( xi, xi+McPar.shape[-1]-1 ) # Need Gsig[0,:] != 0 to capture in x below
    ti = np.flatnonzero( Data[:,-2] ) # Mask indices
    Data[:,xi] = scipy.interpolate.interp1d( ti, Data[ti,xi], axis=0, copy=False, \
        bounds_error=False, fill_value=0, assume_sorted=True)(np.arange(Data.shape[0]))
    # NOTE: copy=False matters, when the interpolation obj is to be used after the fitting data changes.
    assert Data[0,xi].sum()==0, 'Oops! Should be zero!?'
    # if Data[0,xi].sum()!=0: Data[:,xi] = np.nan

    if Par.CropData: Data = Data[ Par.CropData:-Par.CropData ] # ++++
    
    Data[np.abs(Data[:,-1])>9.0,-1] = Par.MaskVal # *** mask outliers
    
    N = int(Par.Tbatch//Par['TR']* Par.TRFS) # 10 minutes
    Data = Data[:Data.shape[0]//N*N,:].reshape(-1,N,Data.shape[-1]) # +++ Data[ NofSections, SectionLen, NX+NY]
    return Data
    

In [ ]:
assert Par['Xrnd'] is None, 'Xrnd not implemented!'


In [ ]:
#< ExTb['Data'] = ExTb[[n for n in ('Phy','McPar','Gsig','Hypno') if n in ExTb.columns]].apply(mkData,'columns') # +++ mk Data
ExTb['Data'] = ExTb.apply(mkData,'columns') # +++ mk Data (much simpler!)
# ExTb['ExSec'] = ExTb.Data.apply(lambda x: np.arange(x.shape[0]) )
# Par['DataChan'] = ['Card','Resp','Mc0','Mc1','Mc2','Mc3','Mc4','Mc5','McNorm','Mask','Gsig']
Par['DataChan'] = ['Card','Resp','Mc0','Mc1','Mc2','Mc3','Mc4','Mc5','McNorm','Hypno','Mask','Gsig']
ExTb.attr = pd.Series(Par) # attach (Ex)Par to ExTb
# ExTb.meth = mkData


In [ ]:
### add col. McDifMax
ExTb['McDifMax'] = ExTb['Data'].apply(lambda x: x[...,8].max(-1))

In [ ]:
### add col. ValSec = numeric tag for each data section
# HOWTO remove Pandas columns whether or not they exist:
ExTb.drop(columns=['TEST','ValSec'],inplace=True,errors='ignore') # remove preexisting cols.

# HOWTO insert a Pandas column after another by name:
ExTb.insert( ExTb.columns.get_loc('ValTag')+1,'ValSec',None) # insert after ValTag

ExTb['ValSec'] = ExTb.Data.apply(lambda x: np.arange(x.shape[0])==x.shape[0]//2)

# ValSec = ValSec | ValEx | ValSu
# HOWTO do np.logical_or on BOOLEAN arrays:
# IFF operands are BOOLEAN you can use "bitwise" operators &,|,^, instead of the clumsy np.logical_*
# ExTb.ValSec = ExTb.apply(lambda x: x.ValSec | (x.ValTag != False ),'columns')
ExTb.ValSec = ExTb.apply( lambda x: np.logical_or( x.ValSec, x.ValTag != False ),'columns')

In [ ]:
### Redo ValSec labels: 1=val.section, 2=(ValTag!=False, e.g. ValEx), 3=(ValTag==ValSu)
def ExTb_ValSec(ExTb):
    # HOWTO remove Pandas columns whether or not they exist:
    ExTb.drop(columns=['TEST','ValSec'],inplace=True,errors='ignore') # remove preexisting cols.

    # HOWTO insert a Pandas column after another by name:
    ExTb.insert( ExTb.columns.get_loc('ValTag')+1,'ValSec',None) # insert after ValTag
    
    # OLD: ver.1
    # ExTb['ValSec'] = ExTb.Data.apply(lambda x: np.arange(x.shape[0])==x.shape[0]//2)
    # ExTb.ValSec = ExTb.apply( lambda x: np.logical_or( x.ValSec, x.ValTag != False ),'columns')

    ExTb['ValSec'] = ExTb.Data.apply(lambda x: np.arange(x.shape[0])/x.shape[0])
    ExTb.ValSec = ExTb.apply( lambda x: np.fmax( x.ValSec, 2*(x.ValTag != False)), 'columns')
    ExTb.ValSec = ExTb.apply( lambda x: np.fmax( x.ValSec, 3*(x.ValTag == 'ValSu')), 'columns')


ExTb_ValSec( ExTb )

## Expand ExTb into SecTb

In [ ]:
## Expand rows
# SecTb = [ (x.Su,x.Se,x.Ex,n,x.Data[n],x.ValSu,x.ValEx,x.ValSec[n]) for x in ExTb.itertuples(index=False) for n in range(x.Data.shape[0]) ]
# SecTb = pd.DataFrame(SecTb,columns=['Su','Se','Ex','Sec','Data','ValSu','ValEx','ValSec']) # tuples -> DF
SecTb = [ (x.Su, x.Se, x.Ex,n, x.ValTag, x.ValSec[n], x.Data[n]) for x in ExTb.itertuples(index=False) for n in range(x.Data.shape[0]) ]
SecTb = pd.DataFrame(SecTb,columns=['Su','Se','Ex','Sec','ValTag','ValSec','Data']) # tuples -> DF
SecTb = SecTb.set_index(['Su','Se','Ex','Sec'], drop=False, append=False, inplace=False, verify_integrity=True)
SecTb = pd.concat( [SecTb.iloc[:,4:],SecTb.iloc[:,:4]],'columns')
# SecTb


## identify sections with motion

In [ ]:
### SecTb.McDifMax = each sections max. motion derivative
SecTb['McDifMax'] = SecTb['Data'].apply(lambda x: x[...,8].max(-1))


*********************************************************

*****************************************************************
## Compare section signals and spectra
Might employ some sort of (spectral) clustering to identify poor-quality Physio signals?!


******************************************
## Save ExTb

***************************************************************
## Intermezzo: Use Xarray?
Remember, an xr.DataArray consists of an MD np.array [.values], a list of dimension names [.dims] (str), and optionally a dict of index vectors [.coords] corresponding to the [.dims].

Dims+coords can also be passed as a list of tuples [('x',np.arange(10)),...] or PD DF or index objects.

***************************************

****************************************
# EEG Sleep Scores
Just divide the entire EEG duration into 30s intervals and assign them to the sleep scores.
